# Introducing tf.estimator
**Learning Objectives**
  - Understand where the `tf.estimator` module sits in the hierarchy of Tensorflow APIs
  - Understand the workflow of creating a `tf.estimator` model
    1. Create Feature Columns
    2. Create Input Functions
    3. Create Estimator
    4. Train/Evaluate/Predict
  - Understand how to swap in/out different types of Estimators
  
Tensorflow is a hierarchical framework. The further down the hierarchy you go, the more flexibility you have, but that more code you have to write. Generally one starts at the highest level of abstraction. Then if you need additional flexibility drop down one layer.

<img src='assets/TFHierarchy.png' width='50%'>
<sup>(image: https://www.tensorflow.org/guide/premade_estimators)</sup>

In this notebook we will be operating at the highest level of Tensorflow abstraction, using the Estimator API to predict taxifare prices on the sampled dataset we created previously.

In [1]:
import tensorflow as tf
import pandas as pd
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


## 1) Load Raw Data 

First let's download the raw .csv data. These are the same files created in the `create_datasets.ipynb` notebook

In [2]:
!gsutil cp gs://cloud-training-demos/taxifare/small/*.csv .
!ls -l *.csv

Copying gs://cloud-training-demos/taxifare/small/taxi-test.csv...
Copying gs://cloud-training-demos/taxifare/small/taxi-train.csv...              
Copying gs://cloud-training-demos/taxifare/small/taxi-valid.csv...              
| [3 files][ 11.3 MiB/ 11.3 MiB]                                                
Operation completed over 3 objects/11.3 MiB.                                     
-rw-r--r-- 1 root root 1867645 Jan  6 16:11 taxi-test.csv
-rw-r--r-- 1 root root 8289592 Jan  6 16:11 taxi-train.csv
-rw-r--r-- 1 root root 1737393 Jan  6 16:11 taxi-valid.csv


Because the files are small we can load them into in-memory Pandas dataframes.

In [3]:
df_train = pd.read_csv('./taxi-train.csv')
df_valid = pd.read_csv('./taxi-valid.csv')
df_test = pd.read_csv('./taxi-test.csv')

CSV_COLUMN_NAMES = list(df_train)
print(CSV_COLUMN_NAMES)

FEATURE_NAMES = CSV_COLUMN_NAMES[1:] # all but first column
LABEL_NAME = CSV_COLUMN_NAMES[0] # first column

['fare_amount', 'dayofweek', 'hourofday', 'pickuplon', 'pickuplat', 'dropofflon', 'dropofflat', 'passengers']


## 2) Create Feature Columns

Feature columns make it easy to perform common type of feature engineering on your raw data. For example you can one-hot encode categorical data, create feature crosses, embeddings and more. We'll cover these later in the course, but if you want to a sneak peak browse the official TensorFlow [feature columns guide](https://www.tensorflow.org/guide/feature_columns).

In our case we won't do any feature engineering. However we still need to create a list of feature columns because the Estimator we will use requires one. To specify the numeric values should be passed on without modification we use `tf.feature_column.numeric_column()`

We use a [python list comprehension](https://www.pythonforbeginners.com/basics/list-comprehensions-in-python) to create the list of feature columns, which is just an elegant alternative to a for loop.

In [4]:
feature_columns = [tf.feature_column.numeric_column(k) for k in FEATURE_NAMES]
feature_columns

[_NumericColumn(key='dayofweek', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='hourofday', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='pickuplon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='pickuplat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='passengers', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## 3) Define Input Function

Now that your estimator knows what type of data to expect and how to intepret it, you need to actually pass the data to it! This is the job of the input function.

The input function returns a new batch of (features, label) tuples each time it is called by the Estimator.

- features: A python dictionary. Each key is a feature column name and its value is the tensor containing the data for that feature
- label: A Tensor containing the labels

So how do we get from our current Pandas dataframes to (features, label) tuples that return one batch at a time?

The `tf.data` module contains a collection of classes that allows you to easily load data, manipulate it, and pipe it into your model. https://www.tensorflow.org/guide/datasets_for_estimators

In [5]:
def train_input_fn(df, batch_size=128):
    #1. Convert dataframe into correct (features,label) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices((dict(df[FEATURE_NAMES]), df[LABEL_NAME]))
    
    # Note:
    # If we returned now, the Dataset would iterate over the data once  
    # in a fixed order, and only produce a single element at a time.
    
    #2. Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
   
    return dataset

def eval_input_fn(df, batch_size=128):
    #1. Convert dataframe into correct (features,label) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices((dict(df[FEATURE_NAMES]), df[LABEL_NAME]))
    
    #2.Batch the examples.
    dataset = dataset.batch(batch_size)
   
    return dataset

def predict_input_fn(df, batch_size=128):
    #1. Convert dataframe into correct (features) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices(dict(df[FEATURE_NAMES])) # no label

    #2.Batch the examples.
    dataset = dataset.batch(batch_size)
   
    return dataset

## 4) Choose Estimator

Tensorflow has several premade estimators for you to choose from:

- LinearClassifier/Regressor
- BoostedTreesClassifier/Regressor
- DNNClassifier/Regressor
- DNNLinearCombinedClassifier/Regressor

If none of these meet your needs you can implement a custom estimator using `tf.Keras`. We'll cover that later in the course.

For now we will use the premade LinearRegressor. To instantiate an estimator simply pass it what feature columns to expect and specify an directory for it to output checkpoint files to.

In [6]:
OUTDIR = 'taxi_trained'

model = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(tf_random_seed=1) # for reproducibility
)

INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_train_distribute': None, '_task_id': 0, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fea5a1ccba8>, '_evaluation_master': '', '_model_dir': 'taxi_trained', '_save_checkpoints_secs': 600, '_task_type': 'worker', '_experimental_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': None, '_device_fn': None, '_tf_random_seed': 1, '_num_worker_replicas': 1, '_protocol': None, '_eval_distribute': None, '_is_chief': True, '_service': None}


## 5) Train

Simply invoke the estimator's `train()` function. Specify the `input_fn` which tells it how to load in data, and specify the number of steps to train for.

By default estimators check the output directory for checkpoint files before beginning training, so it can pickup where it last left off. To prevent this we'll delete the output directory before starting training each time.

In [7]:
%%time
tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model.train(
    input_fn=lambda:train_input_fn(df_train), 
    steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 23887.37, step = 1
INFO:tensorflow:global_step/sec: 140.367
INFO:tensorflow:loss = 5602.9077, step = 101 (0.715 sec)
INFO:tensorflow:global_step/sec: 160.507
INFO:tensorflow:loss = 12591.359, step = 201 (0.625 sec)
INFO:tensorflow:global_step/sec: 153.863
INFO:tensorflow:loss = 8043.271, step = 301 (0.651 sec)
INFO:tensorflow:global_step/sec: 170.799
INFO:tensorflow:loss = 7220.1523, step = 401 (0.582 sec)
INFO:tensorflow:Saving checkpoints for 500 into taxi_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 6319.714.
CPU times: user 6.22 s, sys: 1.48 s, total: 7.7 s
Wall time: 8.22 s


## 6) Evaluate

Estimators similarly have an `evaluate()` function. In this case we don't need to specify the number of steps to train because we didn't tell our input function to repeat the data. Once the input function reaches the end of the data evaluation will end. 

Loss is reported as MSE by default so we take the square root before printing.

In [8]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn=lambda:eval_input_fn(df))
  print('RMSE on dataset = {}'.format(metrics['average_loss']**.5))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-06-16:11:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-06-16:11:13
INFO:tensorflow:Saving dict for global step 500: average_loss = 86.40852, global_step = 500, label/mean = 11.2297125, loss = 11042.246, prediction/mean = 10.282517
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: taxi_trained/model.ckpt-500
RMSE on dataset = 9.295618514094171


RMSE of 9.30 is  worse than our rules based benchmark (RMSE of $7.46). However given that we haven't done any feature engineering or hyperparameter tuning, and we're training on a small dataset using a simple linear model, we shouldn't yet expect good performance. 

The goal at this point is to demonstrate the mechanics of the Estimator API. In subsequent notebooks we'll improve on the model.

## 7) Predict

In [ ]:
predictions = model.predict(input_fn=lambda:predict_input_fn(df_test))
for items in predictions:
  print(items)

Further evidence of the primitiveness of our model, it predicts almost the same amount for every trip!

## 8) Change Estimator Type

One of the payoffs for using the Estimator API is we can swap in a different model type with just a few lines of code. Let's try a DNN. Note how now we need to specify the number of neurons in each hidden layer.

In [10]:
%%time
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) 
model = tf.estimator.DNNRegressor(
    hidden_units = [10,10], # specify neural architecture
    feature_columns = feature_columns, 
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(tf_random_seed=1)
)
model.train(
    input_fn=lambda:train_input_fn(df_train), 
    steps=500)
print_rmse(model, df_valid)

INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_train_distribute': None, '_task_id': 0, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fea58174f28>, '_evaluation_master': '', '_model_dir': 'taxi_trained', '_save_checkpoints_secs': 600, '_task_type': 'worker', '_experimental_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': None, '_device_fn': None, '_tf_random_seed': 1, '_num_worker_replicas': 1, '_protocol': None, '_eval_distribute': None, '_is_chief': True, '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_in

Our performance is about the same at 9.26. This illustrates an important tenant of machine learning: A more complex model can't outrun bad data. 

Currently since we're not doing any feature engineering our input data has very little signal to learn from, so using a DNN doesn't help.

## Results Summary

- Rules Based Benchmark: RMSE 7.46
- Linear Model: RMSE 9.30
- DNN Model: RMSE 9.26

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License